In [2]:
! pip install requests
import requests


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import requests

url = 'https://youtube.com/watch?v=9bZkp7q19f0'  # Example URL, replace with the desired one
response = requests.get(url)
# Check if the request was successful
print("successful" if response.status_code == 200 else "Page not found" if response.status_code == 404 else "Something went wrong")

successful


In [ ]:
response.raise_for_status()  # Throws error for bad requests

In [ ]:
url = "https://api.github.com/users/octocat"
response = requests.get(url)

data = response.json()
print(data)
print(data['name'])        # The Octocat
print(data['public_repos'])  # e.g., 8

{'login': 'octocat', 'id': 583231, 'node_id': 'MDQ6VXNlcjU4MzIzMQ==', 'avatar_url': 'https://avatars.githubusercontent.com/u/583231?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/octocat', 'html_url': 'https://github.com/octocat', 'followers_url': 'https://api.github.com/users/octocat/followers', 'following_url': 'https://api.github.com/users/octocat/following{/other_user}', 'gists_url': 'https://api.github.com/users/octocat/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/octocat/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/octocat/subscriptions', 'organizations_url': 'https://api.github.com/users/octocat/orgs', 'repos_url': 'https://api.github.com/users/octocat/repos', 'events_url': 'https://api.github.com/users/octocat/events{/privacy}', 'received_events_url': 'https://api.github.com/users/octocat/received_events', 'type': 'User', 'user_view_type': 'public', 'site_admin': False, 'name': 'The Octocat', 'company': '@github', 

In [30]:
url = "https://pokeapi.co/api/v2"  # Example URL for a Pokémon API
response = requests.get(url)

def pokemon_info(name):
    url = f"https://pokeapi.co/api/v2/pokemon/{name}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

pokemon_name = "raichu"  # Example Pokémon name, replace with the desired one
response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{pokemon_name}")

if response.status_code == 200:
    data = response.json()
    print(f'Name:', data['name'])
    print(f'Height:', data['height'])
    print(f'Weight:', data['weight'])
    print(f'Id:', data['id'])
    print(f"Pokemon data extracted successfully!")
else:
    print(f"Error: Pokémon '{pokemon_name}' not found (status code {response.status_code})")
    print(f'failed to extract Pokémon data for {pokemon_name}')

Name: raichu
Height: 8
Weight: 300
Id: 26
Pokemon data extracted successfully!


In [25]:
url = 'https://in.pinterest.com/pin/44050902599444305/'
response = requests.get(url)

with open('image.png', 'wb') as f:
    f.write(response.content)


In [ ]:
print(response.headers)
print(response.cookies)

{'Content-Type': 'application/xml', 'Content-Length': '243', 'Connection': 'keep-alive', 'X-Pinterest-Cache-Status-v2': 'Miss', 'AKAMAI-GRN': '0.ee4c6168.1751788355.12000fe6', 'Vary': 'Origin', 'X-CDN': 'akamai', 'Alt-Svc': 'h3=":443"; ma=604800'}
<RequestsCookieJar[]>


In [35]:
from requests.auth import HTTPBasicAuth

response = requests.get('https://httpbin.org/basic-auth/user/pass',
                        auth=HTTPBasicAuth('user', 'pass'))
print(response.status_code)
print(response.json())

200
{'authenticated': True, 'user': 'user'}


 1. requests.get()
Used to get (fetch) data from a URL.

In [ ]:
import requests

response = requests.get('https://api.github.com')

print(response.status_code)   # HTTP status code, like 200 (OK)
print(response.text)          # Response content as a string


200
{'args': {}, 'data': '', 'files': {}, 'form': {'password': '12345', 'username': 'pratham'}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '31', 'Content-Type': 'application/x-www-form-urlencoded', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.3', 'X-Amzn-Trace-Id': 'Root=1-686a349a-2380170b347951fb77beee4f'}, 'json': None, 'origin': '103.87.31.207', 'url': 'https://httpbin.org/post'}


2. requests.post()
Used to send data (like login info, forms) to a server.

In [ ]:
data = {"name":"pratham", "age": 20}
response  = requests.post('https://httpbin.org/post', json=data)
print(data)
print()
print(f'The connection is successful {response.status_code}' if response.status_code == 200 else 'failed')
print(response.json()) 

{'name': 'pratham', 'age': 20}

The connection is successful 200
{'args': {}, 'data': '{"name": "pratham", "age": 20}', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '30', 'Content-Type': 'application/json', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.3', 'X-Amzn-Trace-Id': 'Root=1-686a365f-0e115af670e29fac26c4f5d8'}, 'json': {'age': 20, 'name': 'pratham'}, 'origin': '103.87.31.207', 'url': 'https://httpbin.org/post'}


 3. requests.put()
Used to update existing data on the server.

In [56]:
data = {'name': 'Prathamesh Talele'}

response = requests.put('https://httpbin.org/put', data=data)

print(response.status_code)
print(response.text)

200
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "Prathamesh Talele"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "22", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.32.3", 
    "X-Amzn-Trace-Id": "Root=1-686a3a38-452a86a262976b594f1b114e"
  }, 
  "json": null, 
  "origin": "103.87.31.207", 
  "url": "https://httpbin.org/put"
}



In [ ]:
response = requests.post('https://httpbin.org/delay/3', timeout=2)
print(response.status_code)  # This will raise a Timeout exception if the request takes too long